# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [22]:
import pandas as pd
import numpy as np

import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
# read data into DF
homepage_actions = pd.read_csv('homepage_actions.csv')
# inspect first five rows
homepage_actions.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [6]:
homepage_actions.id.describe()

count      8188.000000
mean     564699.749878
std      219085.845672
min      182988.000000
25%      373637.500000
50%      566840.500000
75%      758078.000000
max      937217.000000
Name: id, dtype: float64

In [8]:
len(homepage_actions)

8188

In [12]:
experiment = homepage_actions[ homepage_actions['group'] == 'experiment' ]
control = homepage_actions[ homepage_actions['group'] == 'control' ]

In [26]:
experiment['action'].value_counts()

view     2996
click     928
Name: action, dtype: int64

In [27]:
control['action'].value_counts()

view     3332
click     932
Name: action, dtype: int64

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

**My Analysis**

Since we want to compare the ratio of views to clicks for two samples, without specifying an underlying theoretical distribution, we can use a chi squared goodness of fit test.

In [34]:
stats.chisquare(f_obs=[2996, 928], f_exp=[3332, 932])
# f_obs is the experiment group views & clicks
# f_exp is the control group views & clicks ('expected' proportions)

Power_divergenceResult(statistic=33.89952032315072, pvalue=5.8033006040559e-09)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [45]:
#Your code here
expected_clicks = (932/(3332+932))*len(homepage_actions)
expected_clicks

1789.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [42]:
#Your code here

# first, determine standard deviation np(1-p)
n = len(homepage_actions) # number of Bernoulli trials
p = 932/(3332+932) # expected value of Bernoulli trial (average of 932 1s and 3332 0s)
stddev = n*p*(1-p)

abs(928 - expected_clicks)/stddev

0.05027879385597969

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.